In [1]:
import glob
import numpy as np
import os
import matplotlib.pyplot as plt
import re
import pandas as pd
import nibabel as nib

In [ ]:
current_dir = !pwd
current_dir = current_dir[0]
main_dir = "/".join(current_dir.split("/")[:-1])
main_dir

In [ ]:
subs = glob.glob(f'{main_dir}/QC_Fastsurfer/sub-*/seg_mask.nii.gz')
print (f"n if sub in Fastsurfer: {len(subs)}")
fs_mask = main_dir + 'FREESURFER_SEG/{}.reconall/seg_mask.nii.gz'
fs_mask

In [ ]:
fs = [sub.split("/")[-2] for sub in subs]

In [ ]:
def dice_score(a1, a2):
    a1 = a1.astype(bool)
    a2 = a2.astype(bool)
    intersection =  (a1 & a2).sum()
    return 2*intersection/(a1.sum()+a2.sum())

In [ ]:
our_map={
 0: "ALL",
 1: 'Frontal',
 2: 'Hippocampus',
 3: 'Frontal_Parietal',
 4: 'Parietal',
 5: 'Insula',
 6: 'Cingulate',
 7: 'Occipital',
 8: 'BasalGanglia',
 9: 'Thalamus',
 10: 'Cerebellum',
 11: 'CorpusCallosum',
 12: 'Ventricles',
 13: 'VentralDC',
 14: 'WM',
 15: 'Brainstem',
 16: 'Temporal'}

In [ ]:
d = dict()
for n,i in enumerate(subs):
    fs_sub = fs_mask.format(fs[n])
    try:
        d[fs[n]] = dict()
        fs_sub = fs_mask.format(fs[n])
        a1 = nib.load(i).get_fdata().round()
        a2 = nib.load(fs_sub).get_fdata().round()
        for j in np.unique(a1):
            dsc = dice_score(np.where(a1==j,1,0),np.where(a2==j,1,0))
            d[fs[n]][our_map[int(j)]] = dsc
    except Exception as e:
        # Exception handling
        print("An exception occurred:", e)

In [ ]:
d

In [ ]:
df = pd.DataFrame.from_dict(d, orient="index")
df.to_csv("DiceScores.csv")

An exception occurred: No such file or no access: '/home/ibrazug/Dokumente/KindersegV2/Ibra/martin/QC_Freesurfer/sub-k14580.reconall/seg_mask.nii.gz'
An exception occurred: No such file or no access: '/home/ibrazug/Dokumente/KindersegV2/Ibra/martin/QC_Freesurfer/sub-k40710.reconall/seg_mask.nii.gz'

In [ ]:
subs = glob.glob('/home/ibrazug/Dokumente/KindersegV2/Ibra/hbn/QC_Fastsurfer/sub-*/seg_mask.nii.gz')

In [ ]:
subs = glob.glob('/home/ibrazug/Dokumente/KindersegV2/Ibra/hbn/QC_Freesurfer/sub-*/T1.nii.gz')

no-seg + seg + 

In [ ]:
roi = "Thalamus"
x = []
y = []
for i in fs:
    sub = d[i]
    score = sub[roi]
    
        
    x.append(i)
    y.append(score)
                    
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])

ax.bar(x,y)
ax.set_ylabel('Scores')

ax.set_xlabel('Subjects')
ax.set_title(f'Dice Scores for {roi}')
plt.ylim([0.7, 1])
plt.show()

In [ ]:
d

In [ ]:
for i in range(0, len(our_map)):
    roi = our_map[i]
    x = []
    y = []
    for i in fs:
        sub = d[i]
        score = sub[roi]
        x.append(i)
        y.append(score)

    fig = plt.figure()
    ax = fig.add_axes([0.5,0,1,1])

    ax.bar(x,y)
    ax.set_ylabel('Scores')
    ax.set_xlabel('Subjects')
    ax.set_title(f'Dice Scores for {roi}')
    plt.ylim([0.6, 1])
    plt.show()

In [ ]:
len(our_map)